# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 20902082


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/88 [00:00<?, ?it/s]

Rendering models:   2%|▏         | 2/88 [00:02<02:03,  1.43s/it]

Rendering models:   5%|▍         | 4/88 [00:04<01:45,  1.26s/it]

Rendering models:   7%|▋         | 6/88 [00:07<01:46,  1.30s/it]

Rendering models:   8%|▊         | 7/88 [00:07<01:17,  1.05it/s]

...

Rendering models:  88%|████████▊ | 77/88 [00:14<00:01,  8.82it/s]

Rendering models:  93%|█████████▎| 82/88 [00:15<00:00, 11.46it/s]

Rendering models:  97%|█████████▋| 85/88 [00:15<00:00, 12.40it/s]

Rendering models:  99%|█████████▉| 87/88 [00:15<00:00, 13.65it/s]

kayleebug2017                         0.002086
not-logged-in-a99d366ea300f07366a6    0.041025
Drew_Farnsworth                       0.001262
Thedunkmasta                          0.001960
Thedunkmasta                          0.001414
not-logged-in-ab387df784182aa5a555    0.001185
not-logged-in-ea8ed61a7274cfb6a43b    0.002314
nothanku                              0.001032
esazure                               0.009243
not-logged-in-1124b2639e37eb70f4c6    0.000488
Spinne1                               0.001296
Paddy_van_CH                          0.000521
msalin03                              0.000586
msross                                0.000553
clizardo                              0.000730
mkirms                                0.000522
mchaffin                              0.000475
djcameron                             0.000470
stgomes                               0.000492
wthammas                              0.000654
atinkess                              0.001134
not-logged-in

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())